In [40]:
from azureml.core import Workspace
from azureml.core import Experiment

workspace = Workspace.from_config()
datastore = workspace.get_default_datastore()
experiment = Experiment(workspace, name='mask-detector')
compute_target = workspace.compute_targets['v100cluster']

In [59]:
from azureml.core.runconfig import DataReferenceConfiguration 

data_dir = DataReferenceConfiguration(datastore_name=datastore, mode='mount', path_on_datastore='mask-data', path_on_compute='data')
checkpoint_dir = DataReferenceConfiguration(datastore_name=datastore, mode='mount', path_on_datastore='ssd-mobilenet-v2-checkpoint', path_on_compute='checkpoint')

In [60]:
from azureml.core import Environment

env = Environment.from_conda_specification(name='mask-detector', file_path='env.yml')
env.docker.enabled = True
env.docker.base_image = 'mcr.microsoft.com/azureml/base-gpu:openmpi3.1.2-cuda10.0-cudnn7-ubuntu18.04'
env.data_references = {
    'data_dir': data_dir,
    'checkpoint_dir': checkpoint_dir
}

AttributeError: <class 'azureml.core.environment.Environment'> has no attribute data_references

In [56]:
data_dir = datastore.path(path='mask-data', data_reference_name='data')
checkpoint_dir = datastore.path(path='ssd-mobilenet-v2-checkpoint', data_reference_name='checkpoint')

In [57]:
from azureml.core import ScriptRunConfig

src = ScriptRunConfig(source_directory='.',
                      script='train.py',
                      arguments=['--data_dir', data_dir.as_mount(),
                                 '--checkpoint_dir', checkpoint_dir.as_mount()],
                      compute_target=compute_target,
                      environment=env)

In [58]:
experiment.submit(src)

TypeError: Object of type DataReference is not JSON serializable